# Tiền xử lý

In [2]:
import pandas as pd

# Đọc dữ liệu từ file csv
fall_dataset = pd.read_csv("data/data.csv")

# Xóa các hàng có giá trị bị thiếu hoặc trùng lặp
fall_dataset.drop_duplicates(inplace=True)
fall_dataset.dropna(inplace=True)

# Xóa các hàng có giá trị 0 quá 50%
fall_dataset = fall_dataset.loc[(fall_dataset == 0).mean(axis=1) <= 0.5]

In [3]:
fall_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,981,982,983,984,985,986,987,988,989,label
0,0.479957,0.228102,-0.181908,0.482556,0.301600,-0.016666,0.521695,0.401496,-0.137361,0.441226,...,0.439851,0.642006,-0.051918,0.517081,0.780251,0.096666,0.431878,0.785392,0.086423,0
1,0.472596,0.220750,-0.236944,0.474785,0.292848,-0.047951,0.565683,0.319175,-0.159357,0.384925,...,0.437203,0.639434,-0.032616,0.497706,0.798847,0.078273,0.433351,0.776532,0.095198,0
2,0.490978,0.199762,-0.170836,0.491824,0.284425,0.014305,0.551431,0.374622,-0.047448,0.423966,...,0.455252,0.654917,-0.032976,0.496907,0.795347,0.145337,0.459017,0.794917,0.133280,0
3,0.475542,0.208120,-0.187014,0.477825,0.290468,-0.051541,0.544644,0.304297,-0.246757,0.404366,...,0.429451,0.639583,-0.033117,0.499803,0.796165,0.098819,0.406445,0.783485,0.065370,0
4,0.480069,0.207963,-0.183100,0.484249,0.292227,-0.006678,0.531287,0.384569,0.000923,0.431638,...,0.455878,0.653487,-0.052929,0.506503,0.789057,0.110457,0.462058,0.793928,0.118302,0


In [4]:
fall_dataset["label"].value_counts()

label
0    120
1     34
Name: count, dtype: int64

In [5]:
# Tăng số lượng dữ liệu bằng cách nhân bản dữ liệu
for i in range(0, 5):
    fall_dataset = pd.concat([fall_dataset, fall_dataset])  # type: ignore

In [6]:
fall_dataset["label"].value_counts()

label
0    3840
1    1088
Name: count, dtype: int64

In [7]:
X = fall_dataset.drop(["label"], axis=1)
y = fall_dataset["label"]

In [8]:
from imblearn.over_sampling import SMOTE

# # Áp dụng Smote để cần bằng dữ liệu bằng cách tạo thêm các mẫu ví dụ cho lớp thiểu số
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)  # type: ignore

In [9]:
y.value_counts()

label
0    3840
1    3840
Name: count, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

# Tách tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
# Đếm mỗi loại giá trị của mỗi lớp trong tập huấn luyện và tập kiểm tra
print("Số ví dụ trong tập train: %s" % y_train.count())
print("Số ví dụ trong tập test: %s" % y_test.count())

Số ví dụ trong tập train: 6144
Số ví dụ trong tập test: 1536


In [12]:
from sklearn.preprocessing import StandardScaler

# Chuẩn hóa các đặc trưng (tùy chọn, nhưng có thể cải thiện hiệu suất cho một số mô hình)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [13]:
X_test

array([[ 1.62057233,  1.32638381,  0.93355208, ...,  0.04895483,
         0.38377809,  0.59776741],
       [ 0.3013961 , -0.3252133 , -0.05228774, ...,  0.03422597,
         0.16410677,  0.0812613 ],
       [ 0.48079228, -0.20192508,  0.72621781, ...,  0.07575559,
         0.11949168, -0.77499772],
       ...,
       [ 0.30447824, -0.38483289, -0.3937449 , ...,  0.03966798,
         0.17874968,  0.10064526],
       [ 1.78967821,  1.74811477, -0.53941973, ...,  1.40883449,
         0.54804845, -1.78960186],
       [ 1.42158891, -0.35740632,  1.38240363, ...,  1.75774141,
         0.42541892, -0.40155619]])

# Vẽ biểu đồ 3D

In [14]:
import plotly.graph_objects as go
import pandas as pd

data_sets = [(X_train, y_train, "Train"), (X_test, y_test, "Test")]
train_colors = ["blue", "red"]
test_colors = ["green", "orange"]
visible_dict = {"Normal": True, "Anomaly": True}
fig = go.Figure()

# Tạo biểu đồ 3D cho tập huấn luyện và tập kiểm tra
for (X, y, name_prefix), colors in zip(data_sets, [train_colors, test_colors]):
    balanced_data = pd.DataFrame(
        {"x": X[:, 0], "y": X[:, 1], "z": X[:, 2], "anomaly": y}
    ).sample(n=min(50000, len(X)), random_state=42)

    for anomaly_label, color in zip(range(2), colors):
        scatter = go.Scatter3d(
            x=balanced_data[balanced_data["anomaly"] == anomaly_label]["x"],
            y=balanced_data[balanced_data["anomaly"] == anomaly_label]["y"],
            z=balanced_data[balanced_data["anomaly"] == anomaly_label]["z"],
            mode="markers",
            marker=dict(size=4, color=color, opacity=0.1),
            name=f'{name_prefix} Data: {"Normal" if anomaly_label == 0 else "Anomaly"}',
            visible=visible_dict.get("Normal", True)
            if anomaly_label == 0
            else visible_dict.get("Anomaly", True),
        )
        fig.add_trace(scatter)

# Thêm nút để chuyển đổi giữa các lớp
buttons = [
    dict(
        label="Normal",
        method="update",
        args=[
            {
                "visible": [
                    True,
                    False,
                    True,
                    False,
                    visible_dict["Anomaly"],
                    visible_dict["Anomaly"],
                ]
            }
        ],
    ),
    dict(
        label="Anomaly",
        method="update",
        args=[
            {
                "visible": [
                    False,
                    True,
                    False,
                    True,
                    visible_dict["Normal"],
                    visible_dict["Normal"],
                ]
            }
        ],
    ),
]

# Cập nhật và hiển thị biểu đồ
fig.update_layout(updatemenus=[dict(type="buttons", showactive=False, buttons=buttons)])
fig.show()

# Thuật toán học có giám sát (Supervised Learning Algorithms)

In [15]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

supervised_learning_models = {
    "KNN": KNeighborsClassifier(n_neighbors=10),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
}

In [16]:
from sklearn.model_selection import GridSearchCV

# Huấn luyện mô hình
train_models = {}
for name, classifier in supervised_learning_models.items():
    model = GridSearchCV(classifier, param_grid={})
    model.fit(X_train, y_train)
    train_models[name] = model

In [17]:
# Đánh giá mô hình
for name, model in train_models.items():
    # Tạo dự đoán trên tập kiểm tra X_test
    y_pred = model.predict(X_test)

    # So sánh dự đoán với giá trị từ tập kết quả y_test
    confusion_mat = pd.crosstab(
        y_test, y_pred, rownames=["Actual"], colnames=["Predicted"]
    )

    # Lấy các giá trị TP, TN, FP, FN từ confusion matrix
    TP, TN, FP, FN = (
        confusion_mat[1][1],
        confusion_mat[0][0],
        confusion_mat[0][1],
        confusion_mat[1][0],
    )

    # Tính toán các chỉ số đánh giá
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = (2 * precision * recall) / (precision + recall)

    # In kết quả
    print(f"{name}:")
    print(confusion_mat)
    print(
        f"Accuracy: {accuracy*100:.2f}%, Precision: {precision*100:.2f}%, Recall: {recall*100:.2f}%, F1: {f1*100:.2f}%"
    )
    print(f"Time: {model.cv_results_['mean_fit_time'][0]:.2f} (s)\n")

KNN:
Predicted    0    1
Actual             
0          782    0
1            0  754
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.10 (s)

Decision Tree:
Predicted    0    1
Actual             
0          782    0
1            0  754
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.41 (s)

Naive Bayes:
Predicted    0    1
Actual             
0          782    0
1            0  754
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.18 (s)



In [18]:
# Lưu mô hình tốt nhất
import pickle

with open("../model/model.pkl", "wb") as file:
    pickle.dump(train_models["KNN"], file)